In [10]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error

import mlflow

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db

# set the tracking uri
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# set experiment
mlflow.set_experiment("nyc-taxi-experiment")

2024/06/14 08:16:38 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/deployment/mlruns/1', creation_time=1718345798690, experiment_id='1', last_update_time=1718345798690, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
  if filename.endswith('.csv'):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
  elif filename.endswith('.parquet'):
    df = pd.read_parquet(filename)

  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

  df = df[(df.duration >= 1) & (df.duration <= 60)]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
    
  return df

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
def prepare_features(df, dv=None):
  # merge location columns
  df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

  # define predictor and target variables
  categorical = ['PU_DO'] #'PULocationID', 'DOLocationID'
  numerical = ['trip_distance']
  target = 'duration'

  # dataframe to dictionaries for vectorizing
  data_dicts = df[categorical + numerical].to_dict(orient='records')

  if dv:
    X = dv.transform(data_dicts)
  else:
    dv = DictVectorizer()
    X = dv.fit_transform(data_dicts)

  y = df[target].values

  return X, y, dv

In [8]:
X_train, y_train, dv = prepare_features(df=df_train)
X_val, y_val, _ = prepare_features(df=df_val, dv=dv)

In [9]:
with mlflow.start_run():
  mlflow.set_tag('Developer', 'Armand Winant')

  # log the datasets used for training and validation
  mlflow.log_param("training-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
  mlflow.log_param("validation-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

  # set and log model parameter
  alpha = .01
  mlflow.log_param("alpha", alpha)

  # create and train model
  lr = Lasso(alpha=alpha)
  lr.fit(X_train, y_train)

  # compute and log validation error
  y_pred = lr.predict(X=X_val)
  rmse = root_mean_squared_error(y_pred, y_val)
  mlflow.log_metric("rmse", rmse)

In [ ]:
# HYPERPARAMETER TUNING (GRID SEARCH)
def objective(params):
  with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)

    booster = xgb.train(
      params = params,
      dtrain = X_train,
      num_boost = 1000,
      evals = [(y_train, "validation")],
      early_stopping_rounds = 50
    )
    y_pred = booster.predict(X_val)
    rmse = root_mean_squared_error(y_pred, y_val)
    mlflow.log_metric("rmse", rmse)
  
  return { 'loss': rmse, 'status': STATUS_OK }

In [ ]:
search_space = {
  'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
  'objective': 'reg:linear',
  'seed': 42
}

best_result = fmin(
  fn=objective,
  space=search_space,
  algo=tpe.suggest,
  max_evals=50,
  trials=Trials
)